In [25]:
import sys
import django
import os
file_dir = "/Users/mirbilal/Desktop/minsir/"
if file_dir not in sys.path:
    sys.path.insert(0, file_dir)

os.environ["DJANGO_SETTINGS_MODULE"] = "minsirx.settings"
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true" 
os.environ["OPENAI_API_KEY"] = "sk-proj-FZ2v2sIScsxguqHnuT4VT3BlbkFJEgAuDrqMx91v7PG2fhDu"
django.setup()
from apps.email_manager.service_layer.email_linker import ConversationReader


In [50]:


convo_reader = ConversationReader()
convo_reader.extract_conversation()

In [51]:

for a_convo_str, a_convo_data in convo_reader.conversations.items():
    if 'INVOICE FOR INSURANCE COVER' in a_convo_str:
        full_convo = a_convo_data.get_full_convo()
        for a_msg in full_convo:
            print(json.dumps(a_msg, indent=4))
        print("")
        print("------------------------------------")
        print("")


{
    "from": "sales.adeel@adamjeeinsurance.com",
    "to": [
        "feroze.vakil@adamjeeinsurance.com"
    ],
    "date": "May 08, 2023, 9:18 AM",
    "subject": "FW: INVOICE FOR INSURANCE COVER",
    "body": "Dear Feroze bhai,\n\u00a0\nKindly issue cover notes.\n\u00a0\n"
}
{
    "from": "abdul.rashid@adamjeeinsurance.com",
    "to": [
        "shahzad@almuqeettextiles.com",
        "mir.babarali@adamjeeinsurance.com"
    ],
    "date": "May 08, 2023, 9:22 AM",
    "subject": "RE: INVOICE FOR INSURANCE COVER",
    "body": "Please attached invoice ?\n\u00a0\n"
}
{
    "from": "sales.adeel@adamjeeinsurance.com",
    "to": [
        "abdul.rashid@adamjeeinsurance.com",
        "shahzad@almuqeettextiles.com",
        "mir.babarali@adamjeeinsurance.com"
    ],
    "date": "May 08, 2023, 9:24 AM",
    "subject": "RE: INVOICE FOR INSURANCE COVER",
    "body": "Dear Rashid bhai,\n\u00a0\nFYI.\n\u00a0\n"
}
{
    "from": "shahzad@almuqeettextiles.com",
    "to": [
        "mir.babarali@adamj

KeyboardInterrupt: 